In [1]:
# Import everything needed to edit/save/watch video clips
import moviepy 

from moviepy.editor import VideoFileClip
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline



In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [3]:

def average_line(image, lines, y_min, y_max):

    index = 0
    indexLeft     = 0
    indexRight    = 0
    slopeLeftSum  = 0
    slopeRightSum = 0
    m_l_sum = 0 # offset of left side
    m_r_sum = 0
    
    # Average the slope and offset for line parameters
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1)) # calculate the slope
            m     = ((y1 - slope*x1)+(y2 - slope*x2))/2 # average the offset
            index= index +1
            if slope > 0:
                indexLeft   = indexLeft + 1;
                slopeLeftSum = slope + slopeLeftSum
                m_l_sum      = m + m_l_sum
            else:
                indexRight = indexRight + 1
                slopeRightSum = slope + slopeRightSum
                m_r_sum      = m + m_r_sum
                
    thickness        = 10
    color            = (255,0,0)
    thick_line_image = np.copy(image)
    
    # pass the parameters through frames to smooth the results
    global average_left
    global average_right
    global average_m_l
    global average_m_r
    global first_frame_left
    global first_frame_right
    
    # use the number of lines from Hough transform as confidence number
    average_factor_l = 1.0 - min(indexLeft,5.0)/10.0
    # define minimum number of line number from Hough Transform
    line_num_threhold = 3
    
    if indexLeft > line_num_threhold:
        if first_frame_left == 1:
            average_left  = slopeLeftSum/indexLeft
            average_m_l   = m_l_sum/indexLeft
            first_frame_left = 0
        else: 
            average_left  = average_left*average_factor_l + slopeLeftSum/indexLeft*(1.0-average_factor_l)  # Average slope
            average_m_l   = average_m_l *average_factor_l  + m_l_sum/indexLeft*(1.0-average_factor_l)
            
    # if this frame is not sufficient, simply take the results from last frame
    x_min_left = int((y_min-average_m_l)/average_left)
    x_max_left = int((y_max-average_m_l)/average_left)
    cv2.line(thick_line_image,(x_min_left, y_min),(x_max_left,y_max),color,thickness)
    
    # use the number of lines from Hough transform as confidence number
    average_factor_r = 1.0 - min(indexRight,5.0)/10.0   
    if indexRight > line_num_threhold:
        if first_frame_right == 1: # Directly take the value without smoothing for first frame
            average_right  = slopeRightSum/indexRight
            average_m_r    = m_r_sum/indexRight
            first_frame_right = 0
        else:  
            average_right = average_right*average_factor_r + slopeRightSum/indexRight*(1.0-average_factor_r) # Average slope
            average_m_r   = average_m_r*average_factor_r  + m_r_sum/indexRight*(1.0-average_factor_r)
                   
    # if this frame is not sufficient, simply take the results from last frame         
    x_min_right = int((y_min-average_m_r)/average_right)
    x_max_right = int((y_max-average_m_r)/average_right)
    cv2.line(thick_line_image,(x_min_right,y_min),(x_max_right,y_max),color,thickness)  

    line_image = cv2.addWeighted(image, 0.7, thick_line_image, 0.3, 0) 
    
    return line_image

In [4]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # convert the rgb image to gray image
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    # smooth the image using gassian blur
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # apply the Canny transform
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # apply the ROI
    # define the polygon points TBD: can be adjust dynamically
    y_min = 310
    y_max = 540

    point1  = (75,  y_max)
    point2  = (455, y_min)
    point3  = (495, y_min)
    point4  = (910, y_max)
    vertices = np.array([[point1, point2, point3 ,point4]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # Apply Hough transform
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 #1# distance resolution in pixels of the Hough grid
    theta = np.pi/170  #180 # angular resolution in radians of the Hough grid
    threshold = 10     #5 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #10#minimum number of pixels making up a line
    max_line_gap = 2    #1# maximum gap in pixels between connectable line segments
    line_image = np.copy(image)#*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    

    line_image = average_line(image, lines, y_min, y_max)
    
    return line_image



In [5]:
white_output = 'solidYellowLeft_Output.mp4'

average_left  = 0
average_right = 0
average_m_l   = 0
average_m_r   = 0
first_frame_left    = 1
first_frame_right   = 1
clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video solidYellowLeft_Output.mp4
[MoviePy] Writing video solidYellowLeft_Output.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:37<00:00, 22.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidYellowLeft_Output.mp4 

Wall time: 38.8 s
